In [1]:
import numpy as np
from control import frequency_response
from sklearn.pipeline import Pipeline

from sippy_unipi.datasets import load_sample_siso
from sippy_unipi.io import ARMAX
from sippy_unipi.model_selection import GridSearchCV, bic_scorer
from sippy_unipi.plot import plot_bode, plot_response, plot_responses
from sippy_unipi.preprocessing import StandardScaler

seed = 0
np.random.seed(seed)

%load_ext autoreload
%autoreload 2

# ARMAX Example using Information Criterion

In [2]:
n_samples = 401
ts = 1.0
time, Ysim, Usim, g_sys, Yerr, Uerr, h_sys, Y, U = load_sample_siso(
    n_samples, ts, seed=seed
)

In [ ]:
fig = plot_responses(
    time,
    [Usim, Uerr, U],
    [Ysim, Yerr, Y],
    ["u", "e", ["u", "e"]],
)

## Perform system identification from collected data

In [ ]:
scaler = StandardScaler()
model = ARMAX(max_iter=1000)

param_grid = {
    "model__na": list(range(1, 6)),
    "model__nb": list(range(1, 6)),
    "model__nc": list(range(1, 6)),
    "model__theta": list(range(1, 14)),
}

model_search = GridSearchCV(
    Pipeline(steps=[("scaler", scaler), ("model", model)]),
    param_grid,
    scoring=bic_scorer,
)
model_search.fit(Y.reshape(-1, 1), U.reshape(-1, 1))
model_fitted = model_search.best_estimator_

## Check that output of the identified system is consistent

In [5]:
Y_pred_g = model_fitted.predict(Usim.reshape(-1, 1))
Y_pred_h = model_fitted.predict(Uerr.reshape(-1, 1), noise=True)
Y_pred = Y_pred_g + Y_pred_h

In [ ]:
fig = plot_response(
    time,
    [Y, Y_pred],
    Usim,
    legends=[["system", "armax"], ["U"]],
    titles=[
        "Output (identification data)",
        "Input, identification data (Switch probability=0.08)",
    ],
)

# Validation of the identified system: 
## Generate new time series for input and noise

In [7]:
n_samples = 401
ts = 1.0
input_range = (0.5, 1.5)
switch_probability = 0.07
time, Ysimval, Usimval, g_sys, Yerrval, Uerrval, h_sys, Yval, Uval = (
    load_sample_siso(n_samples, ts, input_range, switch_probability, seed=seed)
)

## Check responses are almost equal

In [11]:
Y_pred_g_val = model_fitted.predict(Usimval.reshape(-1, 1))
Y_pred_h_val = model_fitted.predict(Uerrval.reshape(-1, 1), noise=True)
Y_pred_val = Y_pred_g_val + Y_pred_h_val

In [ ]:
fig = plot_response(
    time,
    [Yval, Y_pred_val],
    Usim,
    legends=[["system", "armax"], ["U"]],
    titles=[
        "Output (identification data)",
        "Input, identification data (Switch probability=0.07)",
    ],
)

In [ ]:
W_V = np.logspace(-3, 4, num=701)
for tf in ["G_", "H_"]:
    syss_tfs = [
        locals()[f"{tf.lower()}sys"],
        getattr(model_fitted.steps[-1][1], tf),
    ]
    mags, fis, oms = zip(*[frequency_response(sys, W_V) for sys in syss_tfs])

    fig = plot_bode(
        oms[0],
        mags,
        fis,
        ["system"],
    )